<a href="https://colab.research.google.com/github/yukinaga/image_generation/blob/main/section_4/01_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GANの実装
GANの実装を解説します。  
GANではGeneratorとDiscriminaorを競い合わせるように訓練することで、Generatorは次第に本物の画像にそっくりの画像を生成するようになります。  
Discriminatorの学習に用いる本物の画像には、手書き数字画像のデータセットを使います。  
ノイズから画像が生成される過程を観察した上で、GeneratorとDiscriminaorが均衡することを確かめましょう。  

## 手書き文字画像
GANに用いる訓練用のデータを用意します。    
scikit-learnから、8×8の手書き数字の画像データを読み込んで表示します。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

digits_data = datasets.load_digits()

n_img = 10  # 表示する画像の数
plt.figure(figsize=(10, 4))
for i in range(n_img):
    # 入力画像
    ax = plt.subplot(2, 5, i+1)
    plt.imshow(digits_data.data[i].reshape(8, 8), cmap="Greys_r")
    ax.get_xaxis().set_visible(False)  # 軸を非表示に
    ax.get_yaxis().set_visible(False)
plt.show()

print("データの形状:", digits_data.data.shape)
print("ラベル:", digits_data.target[:n_img])

## 各設定
GANに必要な各設定を行います。  
Generatorに入力するノイズの数はここで設定します。  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

import torch
from torch.utils.data import DataLoader

# -- 各設定値 --
img_size = 8  # 画像の高さと幅
n_noise = 16  # ノイズの数
n_mid_1 = 32  # 中間層のニューロン数
n_mid_2 = 64  # 中間層のニューロン数

eta = 0.001  # 学習係数
epochs = 1001  # 学習回数
interval = 50  # 経過の表示間隔
batch_size = 16

# -- 訓練データ --
digits_data = datasets.load_digits()
x_train = np.asarray(digits_data.data)
x_train = x_train / 15*2-1  # -1から1の範囲
t_train = digits_data.target

x_train = torch.tensor(x_train, dtype=torch.float)
train_dataset = torch.utils.data.TensorDataset(x_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## Generatorの構築
PyTorchによりGeneratorのモデルを構築します。  
出力層の活性化関数には、Discriminatorへの入力を-1から1の範囲にするためにtanhを使います。  

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.mid_1 = nn.Linear(n_noise, n_mid_1)
        self.mid_2 = nn.Linear(n_mid_1, n_mid_2)
        self.out = nn.Linear(n_mid_2, img_size*img_size)

    def forward(self, x):
        x = F.relu(self.mid_1(x))
        x = F.relu(self.mid_2(x))
        x = F.tanh(self.out(x))
        return x

generator = Generator()
generator.cuda()  # GPU対応
print(generator)

## Discriminatorの構築
PyTorchによりDiscriminatorのモデルを構築します。  
出力層の活性化関数には、0から1までの値で本物かどうかを識別するためにsigmoid関数を使います。  

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.mid_1 = nn.Linear(img_size*img_size, n_mid_2)
        self.mid_2 = nn.Linear(n_mid_2, n_mid_1)
        self.out = nn.Linear(n_mid_1, 1)

    def forward(self, x):
        x = F.relu(self.mid_1(x))
        x = F.relu(self.mid_2(x))
        x = F.sigmoid(self.out(x))
        return x

discriminator = Discriminator()
discriminator.cuda()  # GPU対応
print(discriminator)

### 画像の生成
画像を生成して表示するための関数を定義します。  
画像は、訓練済みのGenertorにノイズを入力することで生成されます。  
画像は16×16枚生成されますが、並べて一枚の画像にした上で表示されます。

In [ ]:
# -- 画像を生成して表示 --
def generate_images(i):
    # 画像の生成
    n_rows = 16  # 行数
    n_cols = 16  # 列数
    noise = torch.randn(n_rows * n_cols, n_noise).cuda()
    g_imgs = generator(noise)
    g_imgs = g_imgs/2 + 0.5  # 0-1の範囲にする
    g_imgs = g_imgs.cpu().detach().numpy()

    img_size_spaced = img_size + 2
    matrix_image = np.zeros((img_size_spaced*n_rows, img_size_spaced*n_cols))  # 全体の画像

    #  生成された画像を並べて一枚の画像にする
    for r in range(n_rows):
        for c in range(n_cols):
            g_img = g_imgs[r*n_cols + c].reshape(img_size, img_size)
            top = r*img_size_spaced
            left = c*img_size_spaced
            matrix_image[top : top+img_size, left : left+img_size] = g_img

    plt.figure(figsize=(8, 8))
    plt.imshow(matrix_image.tolist(), cmap="Greys_r")
    plt.tick_params(labelbottom=False, labelleft=False, bottom=False, left=False)  # 軸目盛りのラベルと線を消す
    plt.show()

## 正解数の計算
Discriminatorによる鑑定の正解数を、カウントする関数を定義します。  
Discriminatorの精度の計算に使用します。

In [ ]:
def count_correct(y, t):
    correct = torch.sum((torch.where(y<0.5, 0, 1) ==  t).float())
    return correct.item()

## 学習
構築したGANのモデルを使って、学習を行います。  
Generatorが生成した画像には正解ラベル0、本物の画像には正解ラベル1を与えてDiscriminatorを訓練します。  
その後に、結合したモデルを使ってGeneratorを訓練しますが、この場合の正解ラベルは1になります。  
これらの訓練を繰り返すことで、本物と見分けがつかない手書き文字画像が生成されるようになります。  

In [ ]:
from torch import optim

# 交差エントロピー誤差関数
loss_func = nn.BCELoss()

# Adam
optimizer_gen = optim.Adam(generator.parameters())
optimizer_disc = optim.Adam(discriminator.parameters())

# ログ
error_record_fake = []  # 偽物画像の誤差記録
acc_record_fake = []  # 偽物画像の精度記録
error_record_real = []  # 本物画像の誤差記録
acc_record_real = []  # 本物画像の精度記録

# -- GANの学習 --
generator.train()
discriminator.train()
for i in range(epochs):
    loss_fake = 0  # 誤差
    correct_fake = 0  # 正解数
    loss_real = 0
    correct_real = 0
    n_total = 0  # データの総数（精度の計算に使用）
    for j, (x,) in enumerate(train_loader):  # ミニバッチ（x,）を取り出す

        n_total += x.size()[0]  # バッチサイズを累積

        # ノイズから画像を生成しDiscriminatorを訓練
        noise = torch.randn(x.size()[0], n_noise).cuda()
        imgs_fake = generator(noise)  # 画像の生成
        t = torch.zeros(x.size()[0], 1).cuda()  # 正解は0
        y = discriminator(imgs_fake)
        loss = loss_func(y, t)
        optimizer_disc.zero_grad()
        loss.backward()
        optimizer_disc.step()  # Discriminatorのみパラメータを更新
        loss_fake += loss.item()
        correct_fake += count_correct(y, t)

        # 本物の画像を使ってDiscriminatorを訓練
        imgs_real= x.cuda()
        t = torch.ones(x.size()[0], 1).cuda()  # 正解は1
        y = discriminator(imgs_real)
        loss = loss_func(y, t)
        optimizer_disc.zero_grad()
        loss.backward()
        optimizer_disc.step()  # Discriminatorのみパラメータを更新
        loss_real += loss.item()
        correct_real += count_correct(y, t)

        # Generatorを訓練
        noise = torch.randn(x.size()[0]*2, n_noise).cuda()  # バッチサイズを2倍にする
        imgs_fake = generator(noise)  # 画像の生成
        t = torch.ones(x.size()[0]*2, 1).cuda()  # 正解は1
        y = discriminator(imgs_fake)
        loss = loss_func(y, t)
        optimizer_gen.zero_grad()
        loss.backward()
        optimizer_gen.step()  # Generatorのみパラメータを更新

    loss_fake /= j+1  # 誤差
    error_record_fake.append(loss_fake)
    acc_fake = correct_fake / n_total  # 精度
    acc_record_fake.append(acc_fake)

    loss_real /= j+1  # 誤差
    error_record_real.append(loss_real)
    acc_real = correct_real / n_total  # 精度
    acc_record_real.append(acc_real)

    # 一定間隔で誤差と生成された画像を表示
    if i % interval == 0:
        print ("Epochs:", i)
        print ("Error_fake:", loss_fake , "Acc_fake:", acc_fake)
        print ("Error_real:", loss_real , "Acc_real:", acc_real)
        generate_images(i)

学習が進むにつれて、次第に明瞭な手書き数字画像が形作られていきます。  
GeneratorはDiscriminatorをうまく騙せるように、DiscriminatorはGeneratorに騙されないように、互いに切磋琢磨した結果、本物に近い画像が生成されるようになりました。      
なお、学習がうまく進まない場合もあるので、そのような場合は学習を最初からやり直してみましょう。


### 誤差と正解率の推移
学習中における、誤差と正解率の推移を確認します。  
Discriminatorに本物画像を鑑定させた際の誤差の推移と、偽物画像を鑑定させた際の誤差の推移をグラフに表示します。  
正解率の推移も表示します。  

In [ ]:
# -- 誤差の推移 --
plt.plot(range(len(error_record_fake)), error_record_fake, label="Error_fake")
plt.plot(range(len(error_record_real)), error_record_real, label="Error_real")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

# -- 正解率の推移 --
plt.plot(range(len(acc_record_fake)), acc_record_fake, label="Acc_fake")
plt.plot(range(len(acc_record_real)), acc_record_real, label="Acc_real")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

最初、誤差は大きく変動しますが、ある程度時間が経過するとほぼ動かなくなります。  
偽物画像を入力した際場合、Generatorは誤差を上げようとして、Discriminatorは下げようとするので、一種の均衡が生じています。  
また、本物画像を入力した場合、Generatorは真贋の判定を防ぐように学習するので誤差を上げようとして、Discriminatorはこれを下げようとするので、こちらでも均衡が生じます。  
  
正解率について、Generatorが完璧であれば正解率は0.5になり、Discriminatorが完璧であれば正解率は1.0になるはずです。  
正解率は偽物、本物ともに0.5-1.0の範囲に収まっているので、GeneratorとDiscriminatorの均衡がこちらでも観察されます。  
GeneratorとDiscriminatorが競合するように学習し、その結果生じた均衡のなかで、少しずつ本物らしい画像が形作られていきます。  
なお、乱数の値によっては両者がうまく均衡できない場合もあります。




## 演習
GeneratorとDiscriminatorのバランスを、あえて崩してみましょう。  

コードの以下の箇所に注目します。
```
        # Generatorを訓練
        noise = torch.randn(x.size()[0]*2, n_noise).cuda()  # バッチサイズを2倍にする
```
ここを例えば以下のように変更し、両者のバランスに変更を加えてください。
```
        # Generatorを訓練
        noise = torch.randn(x.size()[0]*4, n_noise).cuda()  # バッチサイズを4倍にする
```
これにより、結果がどのように変化するのか確かめてみましょう。  

